In [1]:
%load_ext autoreload
%autoreload 2

# pip install git2doc
from git2doc import loader
import tiktoken
import polars as pl
import os

In [2]:
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string, allowed_special="all"))
    return num_tokens

In [12]:
# pipeline for get_top_repos() -> pull_code_from_repo()
loader.pipeline_fetch_and_load(
    n_repos=2500,
    last_n_days=365,
    language="python",
    write_batch_size=100,
)

Querying 2500 repos, this will be done in 3 batches
Query: 833 repos on 2022-06-24 -> 2022-10-23
Query: 833 repos on 2022-10-23 -> 2023-02-21
Query: 833 repos on 2023-02-21 -> 2023-06-22
Failed to get repos after 10 pages. Returning 800 repos.

(0) Processing https://github.com/AUTOMATIC1111/stable-diffusion-webui...
(1) Processing https://github.com/hwchase17/langchain...
(2) Processing https://github.com/openai/whisper...
(3) Processing https://github.com/zhayujie/chatgpt-on-wechat...
(4) Processing https://github.com/mouredev/Hello-Python...
(5) Processing https://github.com/fauxpilot/fauxpilot...
(6) Processing https://github.com/ultralytics/ultralytics...
(7) Processing https://github.com/Sygil-Dev/sygil-webui...
(8) Processing https://github.com/brycedrennan/imaginAIry...
(9) Processing https://github.com/carson-katri/dream-textures...
(10) Processing https://github.com/IdreesInc/Monocraft...
(11) Processing https://github.com/chroma-core/chroma...
(12) Processing https://github.

Exception in thread Thread-2388 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\subprocess.py", line 1494, in _readerthread
    buffer.append(fh.read())
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8d in position 170: character maps to <undefined>
Exception in thread Thread-2390 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\voyno\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File 

Failed to pull data from https://github.com/datawhalechina/vced after 3 attempts.
(539) Processing https://github.com/garvita-tiwari/PoseNDF...
(540) Processing https://github.com/TEAM-PYRO-BOTZ/PYRO-RENAME-BOT...
(541) Processing https://github.com/Nikorasu/LiveWhisper...
(542) Processing https://github.com/EISMGard/github-audit-tool...
(543) Processing https://github.com/vLAR-group/DM-NeRF...
(544) Processing https://github.com/Run1e/STRIKER...
(545) Processing https://github.com/AirportR/FullTclash...
(546) Processing https://github.com/sena-nana/nonebot-plugin-novelai...
(547) Processing https://github.com/socfortress/Wazuh-Rules...
(548) Processing https://github.com/mindspore-lab/mindcv...
(549) Processing https://github.com/OpenDriveLab/ST-P3...
(550) Processing https://github.com/gangweiX/IGEV...
(551) Processing https://github.com/mhaskar/ExchangeFinder...
(552) Processing https://github.com/MobileTeleSystems/Ambrosia...
(553) Processing https://github.com/catppuccin/minecraft

In [12]:
def num_tokens_wrapper(string: str) -> int:
    return num_tokens_from_string(string, "cl100k_base")

count_dfs = []
folder = "output_data"
for file in os.listdir(folder):
    if "filedata" in file:
        print(f"Reading {file}")
        df = pl.read_parquet(os.path.join(folder, file))
        
        df = df.with_columns(
            pl.col("page_content").apply(num_tokens_wrapper).alias("num_tokens")
        )
        count_dfs.append(df[['num_tokens', 'file_type']])
df = pl.concat(count_dfs)

In [18]:
df.to_pandas()['num_tokens'].sum()

# 700 million tokens across over 2000 repos

708331274